- #### Transformaciones
    - ##### cache, persist
    - ##### coalesce, repartition
    - ##### broadcast

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

spark = SparkSession.builder \
        .appName("sesion_10") \
        .master("local[*]") \
        .config("spark.sql.adaptive.enabled", "false") \
        .getOrCreate()

In [ ]:
spark.conf.set("spark.sql.adaptive.enabled", "false")

In [ ]:
spark

In [ ]:
base_path = "../../resources/data/tmp/parquet/"
movies_df = spark.read.parquet(base_path + "09/movies")

In [ ]:
movies_df.show(1)

In [ ]:
movies_df.count()

In [ ]:
movies_df.printSchema()

### cache, persist

In [ ]:
id_1_df = spark.range(1, 100000000, 1) \
    .select("id", (f.col("id") * 2).alias("id_mult_2"))

id_2_df = spark.range(1, 100000000, 2) \
    .select("id", (f.col("id") * 3).alias("id_mult_3"))

In [ ]:
## expensive generation of id_join
from pyspark.storagelevel import StorageLevel

id_join = id_1_df.join(id_2_df, ["id"])#.persist(StorageLevel.DISK_ONLY)

In [ ]:
id_join.count()

In [ ]:
result_1 = movies_df \
    .join(id_join, f.col("movie_id") == f.col("id"), "left") \
    .select(f.col("*"), (f.col("year") * (f.col("id") % 2)).alias("other_id"))

In [ ]:
result_2 = result_1 \
    .join(id_join.select(f.col("id")), ["id"], "left_semi")

In [ ]:
result_2.count()